In [1]:
import pandas as pd
import numpy as np
import itertools

from spka import spka

In [2]:
no_sub_A = 1
no_sub_B = 1
no_cat = 2
no_spka_points = 4
spka_conv = 20
t0 = True
order_in_A_B = True
order_in_cat = True

In [3]:
spka1 = spka(no_sub_A, no_sub_B, no_cat, no_spka_points, spka_conv, t0, order_in_A_B, order_in_cat)

In [4]:
spka1.spka_combinations()

[(1,),
 (1, 2, 3),
 (1, 2, 3),
 (1, 2, 3),
 (1, 2, 3),
 (1,),
 (1, 2, 3),
 (1, 2, 3),
 (1, 2, 3),
 (1, 2, 3),
 (1,),
 (1, 2, 3),
 (1, 2, 3),
 (1, 2, 3),
 (1, 2, 3),
 (1,),
 (1, 2, 7),
 (1, 2, 7),
 (1, 2, 7),
 (1, 2, 7),
 (1,),
 (1, 2, 7),
 (1, 2, 7),
 (1, 2, 7),
 (1, 2, 7),
 (1,),
 (1, 2, 7),
 (1, 2, 7),
 (1, 2, 7),
 (1, 2, 7)]

In [5]:
vol_sub_A = 0.1 # mL 0.1, 0.15, 0.1, 0.02, 0.01
vol_sub_B = 0.1 # mL
vol_cat = 0.01 # mL
A0 = 0.1 # M
B0_stand = 0.15 # M
B0_DE = 0.1 # M
C0_Stand = 0.01 #M
C0_DE = 0.02 #M

spka1.spka_volumes(vol_sub_A, vol_sub_B, vol_cat, A0, B0_stand, B0_DE, C0_Stand, C0_DE)

([(0.1,),
  (0.1, 0.1, 0.01),
  (0.08, 0.08, 0.008),
  (0.06, 0.06, 0.006),
  (0.04, 0.04, 0.004),
  (0.1,),
  (0.1, 0.067, 0.01),
  (0.08, 0.053, 0.008),
  (0.06, 0.04, 0.006),
  (0.04, 0.027, 0.004),
  (0.1,),
  (0.1, 0.1, 0.02),
  (0.08, 0.08, 0.016),
  (0.06, 0.06, 0.012),
  (0.04, 0.04, 0.008),
  (0.1,),
  (0.1, 0.1, 0.01),
  (0.08, 0.08, 0.008),
  (0.06, 0.06, 0.006),
  (0.04, 0.04, 0.004),
  (0.1,),
  (0.1, 0.067, 0.01),
  (0.08, 0.053, 0.008),
  (0.06, 0.04, 0.006),
  (0.04, 0.027, 0.004),
  (0.1,),
  (0.1, 0.1, 0.02),
  (0.08, 0.08, 0.016),
  (0.06, 0.06, 0.012),
  (0.04, 0.04, 0.008)],
 {1: 2.28, 2: 1.494, 3: 0.112, 7: 0.112})

# Development

In [44]:
def spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points, t0=True, order_in_A_B=True, order_in_cat=True):
    """
    Params
    --------
    no_sub_A = number of vials in first column (Substrate A)
    no_sub_B = number of vials in second column (Substrate B)
    no_cat = number of vials in third column (Catalyst)
    no_spka_points = number of spka points per profile
    t0 = whether to include a Substrate A t0 at the start of each profile or not
    order_in_A_B = will add kinetic profile to find orders in Substrates A and B
    order_in_cat = will add kinetic profile to find order in Catalyst

    Returns
    --------
    final_combinations = a list containing vial numbers for a full spka run
    """

    # Create lists for each
    sub_A_list = [1 + 4 * i for i in range(no_sub_A)]
    sub_B_list = [2 + 4 * i for i in range(no_sub_B)]
    cat_list = [3 + 4 * i for i in range(no_cat)]

    # Generate all possible combinations
    all_combinations = list(itertools.product(sub_A_list, sub_B_list, cat_list))

    # Reorder combinations so that all combinations with the same elements
    # in sub_B_array and cat_array are grouped together
    combinations = []

    # Iterate through all unique combinations of sub_B_array and cat_array
    # This will go through all sub_A first, then sub_B, then cat
    for c in cat_list:
        for b in sub_B_list:
            for a in sub_A_list:
                combinations.append((a, b, c))

    # Multiply each combination by no_spka_points to create correct number of entries
    # Take into account if we are including a t0 before each profile or not
    spka_combinations = []
    for combination in combinations:
        if t0:
            spka_combinations.append((combination[0],))
        spka_combinations.extend([combination] * no_spka_points)
    
    # Each sequence of vials is the same for each experiment (standard, order in A B, order in Cat)
    # Set up the logic for how many repeats
    repeats = 1
    if order_in_A_B and order_in_cat:
        repeats = 3
    elif order_in_A_B or order_in_cat:
        repeats = 2
    
    # Repeat the combinations accordingly
    final_combinations = []
    group_size = no_spka_points + 1 if t0 else no_spka_points
    for i in range(0, len(spka_combinations), group_size):
        group = spka_combinations[i:i + group_size]
        for _ in range(repeats):
            final_combinations.extend(group)
    
    return final_combinations

In [207]:
thing = spka_combinations(1,1,2,4,False,True,True)
print(len(thing))

24


In [209]:
# NOTE: The number of elements makes sense but the actual values calculated HAVE NOT BEEN CHECKED
def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0_stand, B0_DE, C0_Stand, C0_DE, t0=True, order_in_A_B=True, order_in_cat=True):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (i.e., 10%, 20% steps increasing from 0%)
    A0 = Initial concentration of Substrate A across all experiments
    B0_stand = Initial concentration of Substrate B for standard experiment
    B0_DE = Initial concentration of Substrate B for Different Excess experiment
    C0_Stand = Initial concentration of Catalyst for standard experiment
    C0_DE = Initial concentration of Catalyst for Different Excess experiment
    t0 = include a Substrate A t0 at the start of each profile or not
    order_in_A_B = will add kinetic profile to find orders in Substrates A and B
    order_in_cat = will add kinetic profile to find order in Catalyst

    Returns
    --------
    spka_volumes = A list of volumes (mL) to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining the volume (mL) that has been taken from each vial
    """
    
    # Define no_sub_A, no_sub_B, no_cat, no_spka_points - TO REMOVE
    no_sub_A = 1
    no_sub_B = 1
    no_cat = 2
    no_spka_points = 4

    # Call above function to get a list containing vial numbers for a full spka run
    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points, t0=t0)
    
    # SPKA volume difference currently calculated on % difference between stated initial concentrations for DE experiments
    # Calculate the percentage differences for B and C
    perc_diff_B = (B0_stand - B0_DE) / B0_stand
    perc_diff_C = (C0_Stand - C0_DE) / C0_Stand
    
    # Initialize the list of volumes and volume dictionary
    spka_volumes = []
    combined_volumes = []
    volume_dict = {vial: 0 for combination in final_spka_combination for vial in combination}
    
    # Define a helper function to calculate volumes for each individual kinetic profile
    def calculate_volumes(vol_A, vol_B, vol_C, spka_conv, no_spka_points, t0):
        profile_volumes = []

        # Take into account if we have a t0 at the start of each profile or not
        if t0:
            profile_volumes.append((round(vol_A, 2),))
            start = 1
            total_points = no_spka_points + 1
        else:
            start = 0
            total_points = no_spka_points
        
        for i in range(start, total_points):
            factor = 1 - ((i - start) * spka_conv / 100)
            profile_volumes.append((round(vol_A * factor, 3), round(vol_B * factor, 3), round(vol_C * factor, 3)))
        
        return profile_volumes
    
    # Standard experiment
    standard_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, no_spka_points, t0)
    
    # Different Excess in A and B experiment, only if order_in_A_B is True
    if order_in_A_B:
        sub_B_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B * (1 - perc_diff_B), vol_cat, spka_conv, no_spka_points, t0)
    else:
        sub_B_DE_profile = []
    
    # Different Excess in Catalyst experiment, only if order_in_cat is True
    if order_in_cat:
        cat_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat * (1 - perc_diff_C), spka_conv, no_spka_points, t0)
    else:
        cat_DE_profile = []
    
    # Combine all profiles into spka_volumes list
    combined_volumes.extend(standard_profile)
    if order_in_A_B:
        combined_volumes.extend(sub_B_DE_profile)
    if order_in_cat:
        combined_volumes.extend(cat_DE_profile)
    
    # Duplicate the combined volumes no_sub_A * no_sub_B * no_cat times - this accounts for multiple subsrates and catalysts
    spka_volumes = combined_volumes * (no_sub_A * no_sub_B * no_cat)
    
    # Update volume_dict with total volumes taken
    index = 0
    for profile in spka_volumes:
        if len(profile) == 1:
            vial = final_spka_combination[index % len(final_spka_combination)][0]
            volume_dict[vial] += profile[0]
        else:
            for j, vial in enumerate(final_spka_combination[index % len(final_spka_combination)]):
                volume_dict[vial] += profile[j]
        index += 1
    
    # Round the total volumes in volume_dict to 3 decimal places
    for key in volume_dict:
        volume_dict[key] = round(volume_dict[key], 3)
    
    return spka_volumes, volume_dict

In [212]:
# Example usage
volumes, volume_dict = spka_volumes(0.1, 0.1, 0.01, 30, 0.1, 0.15, 0.1, 0.02, 0.01, 
                                    True, True, True)
print(volumes)
print(volume_dict)
print(len(volumes))
print(len(thing))

[(0.1,), (0.1, 0.1, 0.01), (0.07, 0.07, 0.007), (0.04, 0.04, 0.004), (0.01, 0.01, 0.001), (0.1,), (0.1, 0.067, 0.01), (0.07, 0.047, 0.007), (0.04, 0.027, 0.004), (0.01, 0.007, 0.001), (0.1,), (0.1, 0.1, 0.005), (0.07, 0.07, 0.003), (0.04, 0.04, 0.002), (0.01, 0.01, 0.0), (0.1,), (0.1, 0.1, 0.01), (0.07, 0.07, 0.007), (0.04, 0.04, 0.004), (0.01, 0.01, 0.001), (0.1,), (0.1, 0.067, 0.01), (0.07, 0.047, 0.007), (0.04, 0.027, 0.004), (0.01, 0.007, 0.001), (0.1,), (0.1, 0.1, 0.005), (0.07, 0.07, 0.003), (0.04, 0.04, 0.002), (0.01, 0.01, 0.0)]
{1: 1.92, 2: 1.176, 3: 0.054, 7: 0.054}
30
24


# spka_volumes

In [ ]:
# My outline - not complete
def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0stand, B0DE, C0Stand, C0DE):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (ie 10%, 20% steps increaseing from 0%)?????
    A0 = Initial concententration of Substrate A across all experiments
    B0stand = Initial concentration of Substrate B for standard experiment
    B0DE = Initial concentraiton of Substrate B for Different Excess experiment
    C0stand = Initial concentraiton of Catalyst for standard experiment
    C0DE = Initial concentration of Catalyst for Different Excess experiment

    Returns
    --------
    spka_volumes = A list of volumes to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining how much has been taken from each vial
    """

    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points)

    


In [76]:
# Not sure about this one
def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0stand, B0DE, C0Stand, C0DE):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (i.e., 10%, 20% steps increasing from 0%)
    A0 = Initial concentration of Substrate A across all experiments
    B0stand = Initial concentration of Substrate B for standard experiment
    B0DE = Initial concentration of Substrate B for Different Excess experiment
    C0Stand = Initial concentration of Catalyst for standard experiment
    C0DE = Initial concentration of Catalyst for Different Excess experiment

    Returns
    --------
    spka_volumes = A list of volumes to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining how much has been taken from each vial
    """
    
    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points)
    
    # Calculate the percentage differences for B and C
    perc_diff_B = (B0stand - B0DE) / B0stand
    perc_diff_C = (C0Stand - C0DE) / C0Stand
    
    # Initialize the list of volumes and volume dictionary
    spka_volumes = []
    volume_dict = {'sub_A': 0, 'sub_B': 0, 'cat': 0}

    # Define a helper function to calculate volumes for each profile
    def calculate_volumes(vol_A, vol_B, vol_C, spka_conv, no_spka_points):
        profile_volumes = []
        for i in range(no_spka_points + 1):
            factor = 1 - (i * spka_conv / 100)
            profile_volumes.append((round(vol_A * factor, 2), round(vol_B * factor, 2), round(vol_C, 2)))
        return profile_volumes
    
    # Standard experiment
    standard_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, no_spka_points)
    
    # Different Excess B experiment
    DEB_profile = calculate_volumes(vol_sub_A, vol_sub_B * (1 - perc_diff_B), vol_cat, spka_conv, no_spka_points)
    
    # Different Excess Catalyst experiment
    DEC_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat * (1 - perc_diff_C), spka_conv, no_spka_points)
    
    # Combine all profiles into spka_volumes
    spka_volumes.extend(standard_profile)
    spka_volumes.extend(DEB_profile)
    spka_volumes.extend(DEC_profile)
    
    # Update volume_dict with total volumes taken
    for profile in spka_volumes:
        volume_dict['sub_A'] += profile[0]
        volume_dict['sub_B'] += profile[1]
        volume_dict['cat'] += profile[2]
    
    return spka_volumes, volume_dict

# Example usage
volumes, volume_dict = spka_volumes(0.1, 0.1, 0.01, 20, 0.1, 0.15, 0.1, 0.02, 0.01)
print(volumes)
#print(volume_dict)

[(0.1, 0.1, 0.01), (0.08, 0.08, 0.01), (0.06, 0.06, 0.01), (0.04, 0.04, 0.01), (0.02, 0.02, 0.01), (0.1, 0.07, 0.01), (0.08, 0.05, 0.01), (0.06, 0.04, 0.01), (0.04, 0.03, 0.01), (0.02, 0.01, 0.01), (0.1, 0.1, 0.01), (0.08, 0.08, 0.01), (0.06, 0.06, 0.01), (0.04, 0.04, 0.01), (0.02, 0.02, 0.01)]


In [96]:
# Works but vol_dict is in sub_A etc rather than the vial numbers
def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0stand, B0DE, C0Stand, C0DE):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (i.e., 10%, 20% steps increasing from 0%)
    A0 = Initial concentration of Substrate A across all experiments
    B0stand = Initial concentration of Substrate B for standard experiment
    B0DE = Initial concentration of Substrate B for Different Excess experiment
    C0Stand = Initial concentration of Catalyst for standard experiment
    C0DE = Initial concentration of Catalyst for Different Excess experiment

    Returns
    --------
    spka_volumes = A list of volumes to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining how much has been taken from each vial
    """
    
    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points)
    
    # Calculate the percentage differences for B and C
    perc_diff_B = (B0stand - B0DE) / B0stand
    perc_diff_C = (C0Stand - C0DE) / C0Stand
    
    # Initialize the list of volumes and volume dictionary
    spka_volumes = []
    volume_dict = {'sub_A': 0, 'sub_B': 0, 'cat': 0}

    # Define a helper function to calculate volumes for each profile
    def calculate_volumes(vol_A, vol_B, vol_C, spka_conv, no_spka_points):
        profile_volumes = [(round(vol_A, 2),)]
        for i in range(1, no_spka_points + 1):
            factor = 1 - ((i-1) * spka_conv / 100)
            profile_volumes.append((round(vol_A * factor, 3), round(vol_B * factor, 3), round(vol_C, 3)))
        return profile_volumes
    
    # Standard experiment
    standard_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, no_spka_points)
    
    # Different Excess B experiment
    sub_B_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B * (1 - perc_diff_B), vol_cat, spka_conv, no_spka_points)
    
    # Different Excess Catalyst experiment
    cat_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat * (1 - perc_diff_C), spka_conv, no_spka_points)
    
    # Combine all profiles into spka_volumes
    spka_volumes.extend(standard_profile)
    spka_volumes.extend(sub_B_DE_profile)
    spka_volumes.extend(cat_DE_profile)
    
    # Update volume_dict with total volumes taken
    for profile in spka_volumes:
        if len(profile) == 1:
            volume_dict['sub_A'] += profile[0]
        else:
            volume_dict['sub_A'] += profile[0]
            volume_dict['sub_B'] += profile[1]
            volume_dict['cat'] += profile[2]
    
    # Round the total volumes in volume_dict to 2 decimal places
    for key in volume_dict:
        volume_dict[key] = round(volume_dict[key], 3)
    
    return spka_volumes, volume_dict

In [120]:
# Update for volume_dict to show volume usage per vial
# However, only works for one vial for each sub/cat

def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0_stand, B0_DE, C0_Stand, C0_DE):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (i.e., 10%, 20% steps increasing from 0%)
    A0 = Initial concentration of Substrate A across all experiments
    B0_stand = Initial concentration of Substrate B for standard experiment
    B0_DE = Initial concentration of Substrate B for Different Excess experiment
    C0_Stand = Initial concentration of Catalyst for standard experiment
    C0_DE = Initial concentration of Catalyst for Different Excess experiment

    Returns
    --------
    spka_volumes = A list of volumes to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining how much has been taken from each vial
    """
    
    # Define no_sub_A, no_sub_B, no_cat, no_spka_points
    no_sub_A = 1
    no_sub_B = 1
    no_cat = 1
    no_spka_points = 4
    
    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points)
    
    # Calculate the percentage differences for B and C
    perc_diff_B = (B0_stand - B0_DE) / B0_stand
    perc_diff_C = (C0_Stand - C0_DE) / C0_Stand
    
    # Initialize the list of volumes and volume dictionary
    spka_volumes = []
    volume_dict = {vial: 0 for combination in final_spka_combination for vial in combination}
    
    # Define a helper function to calculate volumes for each profile
    def calculate_volumes(vol_A, vol_B, vol_C, spka_conv, no_spka_points):
        profile_volumes = [(round(vol_A, 2),)]
        for i in range(1, no_spka_points + 1):
            factor = 1 - ((i-1) * spka_conv / 100)
            profile_volumes.append((round(vol_A * factor, 3), round(vol_B * factor, 3), round(vol_C, 3)))
        return profile_volumes
    
    # Standard experiment
    standard_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, no_spka_points)
    
    # Different Excess B experiment
    sub_B_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B * (1 - perc_diff_B), vol_cat, spka_conv, no_spka_points)
    
    # Different Excess Catalyst experiment
    cat_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat * (1 - perc_diff_C), spka_conv, no_spka_points)
    
    # Combine all profiles into spka_volumes
    spka_volumes.extend(standard_profile)
    spka_volumes.extend(sub_B_DE_profile)
    spka_volumes.extend(cat_DE_profile)
    
    # Update volume_dict with total volumes taken
    index = 0
    for profile in spka_volumes:
        if len(profile) == 1:
            vial = final_spka_combination[index][0]
            volume_dict[vial] += profile[0]
        else:
            for j, vial in enumerate(final_spka_combination[index]):
                volume_dict[vial] += profile[j]
        index += 1
    
    # Round the total volumes in volume_dict to 3 decimal places
    for key in volume_dict:
        volume_dict[key] = round(volume_dict[key], 3)
    
    return spka_volumes, volume_dict

In [134]:
# Fix for multiple subs/catalysts
# However, does not handle t0 = False!
def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0_stand, B0_DE, C0_Stand, C0_DE):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (i.e., 10%, 20% steps increasing from 0%)
    A0 = Initial concentration of Substrate A across all experiments
    B0_stand = Initial concentration of Substrate B for standard experiment
    B0_DE = Initial concentration of Substrate B for Different Excess experiment
    C0_Stand = Initial concentration of Catalyst for standard experiment
    C0_DE = Initial concentration of Catalyst for Different Excess experiment

    Returns
    --------
    spka_volumes = A list of volumes to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining how much has been taken from each vial
    """
    
    # Define no_sub_A, no_sub_B, no_cat, no_spka_points
    no_sub_A = 1
    no_sub_B = 1
    no_cat = 2
    no_spka_points = 4
    
    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points, False)
    
    # Calculate the percentage differences for B and C
    perc_diff_B = (B0_stand - B0_DE) / B0_stand
    perc_diff_C = (C0_Stand - C0_DE) / C0_Stand
    
    # Initialize the list of volumes and volume dictionary
    spka_volumes = []
    volume_dict = {vial: 0 for combination in final_spka_combination for vial in combination}
    
    # Define a helper function to calculate volumes for each profile
    def calculate_volumes(vol_A, vol_B, vol_C, spka_conv, no_spka_points):
        profile_volumes = [(round(vol_A, 2),)]
        for i in range(1, no_spka_points + 1):
            factor = 1 - ((i-1) * spka_conv / 100)
            profile_volumes.append((round(vol_A * factor, 3), round(vol_B * factor, 3), round(vol_C, 3)))
        return profile_volumes
    
    # Standard experiment
    standard_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, no_spka_points)
    
    # Different Excess B experiment
    sub_B_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B * (1 - perc_diff_B), vol_cat, spka_conv, no_spka_points)
    
    # Different Excess Catalyst experiment
    cat_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat * (1 - perc_diff_C), spka_conv, no_spka_points)
    
    # Combine all profiles into spka_volumes
    combined_volumes = standard_profile + sub_B_DE_profile + cat_DE_profile
    
    # Duplicate the combined volumes no_sub_A * no_sub_B * no_cat times
    spka_volumes = combined_volumes * (no_sub_A * no_sub_B * no_cat)
    
    # Update volume_dict with total volumes taken
    index = 0
    for profile in spka_volumes:
        if len(profile) == 1:
            vial = final_spka_combination[index % len(final_spka_combination)][0]
            volume_dict[vial] += profile[0]
        else:
            for j, vial in enumerate(final_spka_combination[index % len(final_spka_combination)]):
                volume_dict[vial] += profile[j]
        index += 1
    
    # Round the total volumes in volume_dict to 3 decimal places
    for key in volume_dict:
        volume_dict[key] = round(volume_dict[key], 3)
    
    return spka_volumes, volume_dict

In [ ]:
# Updated to handle t0=false
# However, does not handle order_in_A_B and order_in_cat

# USE THIS IF REMOVING order_in_A_B AND order_in_cat

def spka_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, A0, B0_stand, B0_DE, C0_Stand, C0_DE, t0=True):
    """
    Params
    --------
    vol_sub_A = Maximum volume (mL) of Substrate A stock withdrawn
    vol_sub_B = Maximum volume (mL) of Substrate B stock withdrawn
    vol_cat = Maximum volume (mL) of Catalyst stock withdrawn
    spka_conv = SPKA interval size (i.e., 10%, 20% steps increasing from 0%)
    A0 = Initial concentration of Substrate A across all experiments
    B0_stand = Initial concentration of Substrate B for standard experiment
    B0_DE = Initial concentration of Substrate B for Different Excess experiment
    C0_Stand = Initial concentration of Catalyst for standard experiment
    C0_DE = Initial concentration of Catalyst for Different Excess experiment
    t0 = include a Substrate A t0 at the start of each profile or not

    Returns
    --------
    spka_volumes = A list of volumes to take from each vial for a full SPKA run, an identical partner to that created by spka_combinations
    volume_dict = A dictionary determining how much has been taken from each vial

    TO DO
    --------
    Currently assumes that order_in_A_B and order_in_cat are both true    
    """
    
    # Define no_sub_A, no_sub_B, no_cat, no_spka_points - THESE NEED TO BE MOVED
    no_sub_A = 1
    no_sub_B = 1
    no_cat = 2
    no_spka_points = 4
    
    final_spka_combination = spka_combinations(no_sub_A, no_sub_B, no_cat, no_spka_points, t0=t0)
    
    # Calculate the percentage differences for B and C
    perc_diff_B = (B0_stand - B0_DE) / B0_stand
    perc_diff_C = (C0_Stand - C0_DE) / C0_Stand
    
    # Initialize the list of volumes and volume dictionary
    spka_volumes = []
    volume_dict = {vial: 0 for combination in final_spka_combination for vial in combination}
    
    # Define a helper function to calculate volumes for each profile
    def calculate_volumes(vol_A, vol_B, vol_C, spka_conv, no_spka_points, t0):
        profile_volumes = []
        if t0:
            profile_volumes.append((round(vol_A, 2),))
            start = 1
            total_points = no_spka_points + 1
        else:
            start = 0
            total_points = no_spka_points
        
        for i in range(start, total_points):
            factor = 1 - ((i - start) * spka_conv / 100)
            profile_volumes.append((round(vol_A * factor, 3), round(vol_B * factor, 3), round(vol_C, 3)))
        
        return profile_volumes
    
    # Standard experiment
    standard_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat, spka_conv, no_spka_points, t0)
    
    # Different Excess B experiment
    sub_B_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B * (1 - perc_diff_B), vol_cat, spka_conv, no_spka_points, t0)
    
    # Different Excess Catalyst experiment
    cat_DE_profile = calculate_volumes(vol_sub_A, vol_sub_B, vol_cat * (1 - perc_diff_C), spka_conv, no_spka_points, t0)
    
    # Combine all profiles into spka_volumes
    combined_volumes = standard_profile + sub_B_DE_profile + cat_DE_profile
    
    # Duplicate the combined volumes no_sub_A * no_sub_B * no_cat times
    spka_volumes = combined_volumes * (no_sub_A * no_sub_B * no_cat)
    
    # Update volume_dict with total volumes taken
    index = 0
    for profile in spka_volumes:
        if len(profile) == 1:
            vial = final_spka_combination[index % len(final_spka_combination)][0]
            volume_dict[vial] += profile[0]
        else:
            for j, vial in enumerate(final_spka_combination[index % len(final_spka_combination)]):
                volume_dict[vial] += profile[j]
        index += 1
    
    # Round the total volumes in volume_dict to 3 decimal places
    for key in volume_dict:
        volume_dict[key] = round(volume_dict[key], 3)
    
    return spka_volumes, volume_dict